# Case Study 6 - Neural Networks

__Team Members:__ Amber Clark, Andrew Leppla, Jorge Olmos, Paritosh Rai

# Content
* [Business Understanding](#business-understanding)
    - [Scope](#scope)
    - [Introduction](#introduction)
    - [Methods](#methods)
    - [Results](#results)
* [Data Evaluation](#data-evaluation)
    - [Loading Data](#loading-data) 
    - [Data Summary](#data-summary)
    - [Missing Values](#missing-values)
    - [Feature Removal](#feature-removal)
    - [Exploratory Data Analysis (EDA)](#eda)
* [Model Preparations](#model-preparations)
    - [Sampling & Scaling Data](#sampling-scaling-data)
    - [Proposed Method](#proposed-metrics)
    - [Evaluation Metrics](#evaluation-metrics)
    - [Feature Selection](#feature-selection)
* [Model Building & Evaluations](#model-building)
    - [Sampling Methodology](#sampling-methodology)
    - [Model](#model)
    - [Performance Analysis](#performance-analysis)
* [Model Interpretability & Explainability](#model-explanation)
    - [Examining Feature Importance](#examining-feature-importance)
* [Conclusion](#conclusion)
    - [Final Model Proposal](#final-model-proposal)
    - [Future Considerations, Model Enhancements and Alternative Modeling Approaches](#model-enhancements)

# Business Understanding & Executive Summary <a id='business-understanding'/>

## Objective:


## Introduction:


## Modeling:

### Training and Test Split



### Key Metrics



### Results



### Feature Importance



## Conclusion



## Future Considerations



# Data Evaluation <a id='data-evaluation'>
    

Summarize the data being used in the case using appropriate mediums (charts, graphs, tables); address questions such as: Are there missing values? Which variables are needed (which ones are not)? What assumptions or conclusions are you drawing that need to be relayed to your audience?

## Loading Data <a id='loading-data'>

## Data Summary <a id='data-summary'>

## Missing Values <a id='missing-values'>



## Exploratory Data Analysis (EDA) <a id='eda'>


# Model Preparations <a id='model-preparations'/>

Which methods are you proposing to utilize to solve the problem?  Why is this method appropriate given the business objective? How will you determine if your approach is useful (or how will you differentiate which approach is more useful than another)?  More specifically, what evaluation metrics are most useful given that the problem is a classification one (ex., Accuracy, F1-score, Precision, Recall, AUC, etc.)?

## Sampling & Scaling Data <a id='sampling-scaling-data' />


## Proposed Method <a id='proposed-metrics' />

## Evaluation Metrics <a id='evaluation-metrics' />

### Baseline Model

## Feature Selection <a id='feature-selection' />


# Model Building & Evaluations <a id='model-building'/>

In this case, your primary task is to construct a neural network to detect the existence of new particles and will involve the following steps:

- Construct your neural network's architecture
- Fit your neural network to your training data
- Analyze your model's performance - referencing your chosen evaluation metric (including supplemental visuals and analysis where appropriate)

## Sampling Methodology <a id='sampling-methodology'/>

## Modeling

### Final Model


## Model's Performance Analysis <a id='performance-analysis'/>

## Model Interpretability & Explainability <a id='model-explanation'>

### Final Model Proposal <a id='final-model-proposal'/>

### Examining Feature Importance <a id='examining-feature-importance'/>


# Conclusion <a id='conclusion'>

After all of your technical analysis and modeling; what are you proposing to your audience and why?  How should they view your results and what should they consider when moving forward?  Are there other approaches you'd recommend exploring?  This is where you "bring it all home" in language they understand.

### Future Considerations, Model Enhancements and Alternative Modeling Approaches <a id='model-enhancements'/>

## References